In [1]:
import pandas as pd
df = pd.read_table("guillaume.txt", sep=";")
df.head()

C:\Users\spica\AppData\Local\Temp\ipykernel_12796\2198844990.py:2: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table("guillaume.txt", sep=";")


,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,...,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure
0,A013010004908126703060931,78643044,0,20,2017-02-01 07:32:14,1,0,0,0,551,...,"37,186667890919111","52,076033757361408",0,1,4,4,4,20,0,27134
1,A013011306908024927155000,78643045,0,20,2017-02-01 07:43:37,1,0,0,0,551,...,"48,844716275908937","52,076033757361408",1,2,"1,7976851851851852",4,4,"28,609999999999999","8,6099999999999994",27817
2,A013010002908283134592527,78643046,0,"57,640000000000001",2017-02-01 07:47:38,1,0,0,0,549,...,"73,118279569892479","52,076033757361408",0,1,4,4,4,"57,640000000000001",0,28058
3,A011010002908105209831316,78643047,0,"54,289999999999999",2017-02-01 07:48:48,0,1,1,1,267,...,"110,05692599620494","53,554233554497365",0,1,4,4,4,"54,289999999999999",0,28128
4,A013010041908000125652029,78643048,0,"26,899999999999999",2017-02-01 08:13:27,1,0,0,0,549,...,"45,36831264567185","52,076033757361408",1,1,"1,9971064814814814",4,4,"59,149999999999999","32,25",29607


In [2]:
len(df)

4646774

# Nettoyage 

On supprime la ligne 1956360 qui comporte comme valeur pour chaque colonne l'en-tête

In [3]:
df.drop(1956360, axis=0, inplace=True)

On change le type des variables quantitatives

In [4]:
var_quanti = ["Montant","ScoringFP1","ScoringFP2","ScoringFP3","TauxImpNb_RB","TauxImpNB_CPM","DiffDateTr1","DiffDateTr2","DiffDateTr3","CA3TRetMtt","CA3TR"]
var_quanti_bug = ["VerifianceCPT1","VerifianceCPT2","VerifianceCPT3","EcartNumCheq","NbrMagasin3J","D2CB"]
new_df=df

for i in var_quanti_bug:
    new_df[i] = pd.to_numeric(new_df[i], downcast="float")
# On change le type des données
for i in var_quanti:
    new_df[i] = new_df[i].str.replace(",",".")
    new_df[i] = pd.to_numeric(new_df[i], downcast="float")

In [5]:
import datetime  

date = []
heure = []


for i in new_df["DateTransaction"]:
    split = i.split(" ")
    date.append(split[0])
    heure.append(split[1])

for i in range(0,len(date)):
    date[i] = datetime.datetime.strptime(date[i], '%Y-%m-%d')



In [6]:
new_df["DateTransaction"]=date
new_df["Heure"]=heure

On s'occupe des des valeurs manquantes 

In [7]:
print(new_df.isna().sum())

ZIBZIN                      0
IDAvisAutorisationCheque    0
FlagImpaye                  0
Montant                     0
DateTransaction             0
CodeDecision                0
VerifianceCPT1              0
VerifianceCPT2              0
VerifianceCPT3              0
D2CB                        0
ScoringFP1                  0
ScoringFP2                  0
ScoringFP3                  0
TauxImpNb_RB                0
TauxImpNB_CPM               0
EcartNumCheq                0
NbrMagasin3J                0
DiffDateTr1                 0
DiffDateTr2                 0
DiffDateTr3                 0
CA3TRetMtt                  0
CA3TR                       0
Heure                       0
dtype: int64


Aucune valeur manquantes

On nettoie la colonne FlagImpaye

In [8]:
new_df.FlagImpaye.value_counts()

0    4584151
0      32627
1      29855
1        140
Name: FlagImpaye, dtype: int64

In [9]:
new_df.FlagImpaye=new_df.FlagImpaye.replace(0,"0")
new_df.FlagImpaye=new_df.FlagImpaye.replace(1,"1")
new_df.FlagImpaye.value_counts(normalize=True)

0    0.993545
1    0.006455
Name: FlagImpaye, dtype: float64

In [10]:
train_df=new_df[(new_df["DateTransaction"]>='2017-02-01') & (new_df["DateTransaction"]<='2017-08-31')]
test_df=new_df[(new_df["DateTransaction"]>'2017-08-31')]

In [11]:
Y_train = train_df.FlagImpaye
Y_test = test_df.FlagImpaye
X_train = train_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
X_test = test_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
len(Y_train[Y_train=="1"])

23422

# SAMPLING

In [12]:
train_df=new_df[(new_df["DateTransaction"]>='2017-02-01') & (new_df["DateTransaction"]<='2017-08-31')]
test_df=new_df[(new_df["DateTransaction"]>'2017-08-31')]

In [13]:
Y_train = train_df.FlagImpaye
Y_test = test_df.FlagImpaye
X_train = train_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
X_test = test_df.drop(columns=["FlagImpaye","ZIBZIN","IDAvisAutorisationCheque","CodeDecision","DateTransaction","Heure"])
len(Y_train[Y_train=="1"])

23422

# MACHINE LEARNING

## UNDERSAMPLING : 0.9-0.1 en prenant tous les Y

In [14]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=0.01)
rus.fit(X_train, Y_train)
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train, Y_train)
len(X_train_resampled)

2365622

In [15]:
Y_train_resampled.value_counts(normalize=True)

0    0.990099
1    0.009901
Name: FlagImpaye, dtype: float64

In [24]:
best_f = 0

### Arbre de décision

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

parameters = {'max_depth' : np.arange(start = 16, stop = 21, step = 1) , 
              'min_samples_leaf' : np.arange(start = 2, stop = 4, step = 1),
              'min_samples_split' : np.arange(start = 2, stop = 4, step = 1)}

In [18]:
model_arbre = DecisionTreeClassifier()
f1 = make_scorer(f1_score , average='macro')
model_arbre = GridSearchCV(model_arbre,
                                  parameters,
                                  scoring = 'accuracy',
                                  verbose = 2, 
                                  cv = 2 )
model_arbre.fit(X_train_resampled, Y_train_resampled)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  30.0s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=2; total time=  36.7s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  32.0s
[CV] END max_depth=16, min_samples_leaf=2, min_samples_split=3; total time=  30.2s
[CV] END max_depth=16, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=16, min_samples_leaf=3, min_samples_split=2; total time=  37.9s
[CV] END max_depth=16, min_samples_leaf=3, min_samples_split=3; total time=  38.4s
[CV] END max_depth=16, min_samples_leaf=3, min_samples_split=3; total time=  37.9s
[CV] END max_depth=17, min_samples_leaf=2, min_samples_split=2; total time=  41.7s
[CV] END max_depth=17, min_samples_leaf=2, min_samples_split=2; total time=  40.8s
[CV] END max_depth=17, min_samples_leaf=2, min_samples_split=3; total time=  34.4s
[CV] END max_depth=17, min

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': array([16, 17, 18, 19, 20]),
                         'min_samples_leaf': array([2, 3]),
                         'min_samples_split': array([2, 3])},
             scoring='accuracy', verbose=2)

In [19]:
print("Voici les paramètres du meilleure modèle : " + 
      str(model_arbre.best_estimator_))
print("Voici le "  + str(model_arbre.scorer_) + 
      " du meilleure modèle : " + str(model_arbre.best_score_))

Voici les paramètres du meilleure modèle : DecisionTreeClassifier(max_depth=16, min_samples_leaf=2)
Voici le make_scorer(accuracy_score) du meilleure modèle : 0.990104082562641


In [20]:
Y_pred = model_arbre.predict(X_test)
Y_pred_proba = model_arbre.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,739917,921
obs_1,6264,309


In [25]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

if F>best_f :
    best_model = model_arbre.best_estimator_
    best_f = F

0.07920030757400999


### Adaboost

In [26]:
from sklearn.ensemble import AdaBoostClassifier

model_adaboost = AdaBoostClassifier(best_model)
model_adaboost.fit(X_train_resampled, Y_train_resampled)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=16,
                                                         min_samples_leaf=2))

In [27]:
Y_pred = model_adaboost.predict(X_test)
Y_pred_proba = model_adaboost.predict_proba(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,739608,1230
obs_1,6331,242


In [28]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

0.06016159105034183


### Bagging

In [29]:
from sklearn.ensemble import BaggingClassifier
model_bagging = BaggingClassifier()

model_bagging.fit(X_train_resampled, Y_train_resampled)

BaggingClassifier()

In [30]:
Y_pred = model_bagging.predict(X_test)
from sklearn.metrics import confusion_matrix
mc = pd.DataFrame(confusion_matrix(Y_test,Y_pred), 
                  columns=['pred_0','pred_1'],
                  index=['obs_0','obs_1'])

mc

,pred_0,pred_1
obs_0,739871,967
obs_1,6282,291


In [31]:
TP=mc.iloc[1,1]
FN = mc.iloc[0,1]
FP = mc.iloc[1,0]

F = (2*TP)/((2*TP)+FN+FP)
print(F)

0.07432001021580896
